In [ ]:
# Parameters
"""
:param input_csv_file str: Path to input file
:param model_path str: Path to model files
:param mode str: training or evaluation
"""
input_csv_file = "data/data_train_tokenized.csv"
model_path = 'data/fasttext_model'

In [ ]:
import pandas as pd
import numpy as np
from nltk import ngrams
from pyfasttext import FastText
import tempfile
import os

In [ ]:
df = pd.read_csv(input_csv_file)

In [ ]:
import json
df['data'] = df['data'].apply(lambda s: json.loads(s.replace("'", '"')))

In [ ]:
trigrams_set = [(label, trigram) 
                for (label, words) in df[['targetnames', 'data']].itertuples(index=False, name=None)
                if len(words) >= 3
                for trigram in ngrams(words, 3)]

In [ ]:
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_path = os.path.join(tmp_dir, 'trigrams')
    with open(tmp_path, 'w') as f:
        for label, words in trigrams_set:
            f.write('__label__{} {}\n'.format(label, ' '.join(words)))

    model = FastText()
    model.supervised(input=tmp_path, output=model_path, epoch=10, lr=0.7)